In [14]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import requests
import matplotlib.pyplot as plt

In [2]:
# Define constants
ACCESS_KEY = "qGZH0hMvwG1BphZ5mWdMN8pEljHzZU-Mf2N35ZjHCuA"  # Replace with your Unsplash access key
URL = "https://api.unsplash.com/photos/random"


def fetch_random_images(count=50):
    """
    Fetches random image URLs from Unsplash.

    Args:
        count (int): The number of images to fetch.

    Returns:
        list: A list of random image URLs.
    """

    images = []
    params = {
        "count": count,
        "client_id": ACCESS_KEY,
    }

    try:
        response = requests.get(URL, params=params)
        response.raise_for_status()
        data = response.json()
        images = [img["urls"]["full"] for img in data]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching images: {e}")

    return images

In [3]:
model_id = "openai/clip-vit-base-patch32"

model = CLIPModel.from_pretrained(model_id)
processor = CLIPProcessor.from_pretrained(model_id)

In [4]:
sample_image_urls = fetch_random_images(20)

In [5]:
sample_images = [
    Image.open(requests.get(url, stream=True).raw) for url in sample_image_urls
]

In [26]:
inputs = processor(
    text="abstract",
    images=sample_images,
    return_tensors="pt",
    padding=True,
)

In [40]:
outputs = model(**inputs)

# this is the image-text similarity score
logits_per_image = outputs.logits_per_image.reshape(-1)
probs = logits_per_image.softmax(axis=0)
threshold = 0.1
print(torch.where(probs >= threshold), probs[probs >= threshold])

(tensor([ 3, 14, 15]),) tensor([0.1247, 0.5644, 0.1556], grad_fn=<IndexBackward0>)


In [ ]:
plt.imshow(sample_images[14])
plt.axis(False)

NameError: name 'plt' is not defined